<a href="https://colab.research.google.com/github/arcossci/llm-tools/blob/main/Memory_experimient_Langchain_multiple_approaches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install langchain langchain_community openai google-colab tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


## Experiment 1: ConversationBufferMemory


In [11]:
import os
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from google.colab import userdata

# Load your API key from environment variables
openai_api_key = userdata.get('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

llm = OpenAI(api_key=openai_api_key, temperature=0.7)
# Change memory_key to "history" to match the expected input variable
memory = ConversationBufferMemory(memory_key="history", return_messages=True)
conversation = ConversationChain(llm=llm, memory=memory)

print("ConversationBufferMemory Experiment. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Print all saved memory (the entire conversation history)
saved_memory = conversation.memory.load_memory_variables({})
print("\n=== Saved Memory ===")
# Access the memory using the "history" key
print(saved_memory.get("history", "No memory found."))

ConversationBufferMemory Experiment. Type 'exit' to quit.
You: what is my name?
Bot:  I'm sorry, I don't know your name. Could you please tell me?


KeyboardInterrupt: Interrupted by user

## Experiment 2: ConversationSummaryMemory


In [14]:
import os
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

llm = OpenAI(api_key=openai_api_key, temperature=0.7)
memory = ConversationSummaryMemory(llm=llm, max_token_limit=150)
conversation = ConversationChain(llm=llm, memory=memory)

print("ConversationSummaryMemory Experiment. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Print the conversation summary
saved_memory = conversation.memory.load_memory_variables({})
print("\n=== Saved Memory Summary ===")
print(saved_memory.get("history", "No memory found."))


ConversationSummaryMemory Experiment. Type 'exit' to quit.
You: hola
Bot:  Hello! It's nice to meet you. Did you know that "hola" is a Spanish word that means "hello"? It's often used as a greeting in Spanish-speaking countries.
You: what is my name?
Bot:  I cannot answer that question as I do not have access to your personal information. Is there anything else I can assist you with?


KeyboardInterrupt: Interrupted by user

In [17]:
# prompt: persist the ConversationSummaryMemory at the end of conversation

import json
# ... (your existing code) ...

# ## Experiment 2: ConversationSummaryMemory with Persistence

llm = OpenAI(api_key=openai_api_key, temperature=0.7)
memory = ConversationSummaryMemory(llm=llm, max_token_limit=150)
conversation = ConversationChain(llm=llm, memory=memory)

# Define a file to store the conversation summary
summary_file = "conversation_summary.json"

# Load existing summary if the file exists
if os.path.exists(summary_file):
    try:
        with open(summary_file, "r") as f:
            memory.buffer = json.load(f)  # Restore the buffer from the file
    except json.JSONDecodeError:
        print("Error loading conversation summary. Starting fresh.")


print("ConversationSummaryMemory Experiment. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Save the conversation summary to the file
try:
    with open(summary_file, "w") as f:
        json.dump(memory.buffer, f)
    print(f"\nConversation summary saved to {summary_file}")
except Exception as e:
    print(f"Error saving conversation summary: {e}")


# Print the conversation summary
saved_memory = conversation.memory.load_memory_variables({})
print("\n=== Saved Memory Summary ===")
print(saved_memory.get("history", "No memory found."))


ConversationSummaryMemory Experiment. Type 'exit' to quit.
You: what is my name?
Bot:  I apologize, I do not have access to personal information such as names. However, I am happy to assist with anything else you may need, such as information about your dog, if you have one.
You: do you remeber my dog?
Bot:  I am sorry, but I do not have access to personal information such as your name or your dog's name. However, I can provide information about different breeds of dogs or tips for taking care of them. Would you like me to assist you with that?


KeyboardInterrupt: Interrupted by user

## Experiment 3: ConversationBufferWindowMemory


In [12]:
import os
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

# Load your API key from environment variables
openai_api_key = userdata.get('OPENAI_API_KEY')
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables.")

llm = OpenAI(api_key=openai_api_key, temperature=0.7)
memory = ConversationBufferWindowMemory(k=2)  # Keeps only the last 2 exchanges
conversation = ConversationChain(llm=llm, memory=memory)

print("ConversationBufferWindowMemory Experiment. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Print only the last 2 interactions
saved_memory = conversation.memory.load_memory_variables({})
print("\n=== Saved Memory (Last 2 Exchanges) ===")
print(saved_memory.get("history", "No memory found."))


<ipython-input-12-8f8344fc9c36>:12: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2)  # Keeps only the last 2 exchanges


ConversationBufferWindowMemory Experiment. Type 'exit' to quit.
You: hello i am jeff
Bot:  Hello Jeff, it's nice to meet you! Is there anything specific you would like to talk about? I have access to a wide range of information and can provide details on many topics.
You: my name is...
Bot:  I apologize, it seems I have mistaken your name. What is your name?
You: jeff
Bot:  Ah, my apologies for the confusion. It seems I was correct in addressing you as Jeff earlier. Is there something you would like to discuss, Jeff?
You: what is my name?
Bot:  Your name is Jeff.


KeyboardInterrupt: Interrupted by user

In [20]:
# prompt: persist the ConversationBufferWindowMemory at the conversation's end

import os
import json

# ... (your existing code) ...

# ## Experiment 3: ConversationBufferWindowMemory with Persistence

# Load your API key from environment variables
# ... (your existing code) ...

memory = ConversationBufferWindowMemory(k=10)
conversation = ConversationChain(llm=llm, memory=memory)

# Define a file to store the conversation history
history_file = "conversation_history.json"

# Load existing history if the file exists
if os.path.exists(history_file):
    try:
        with open(history_file, "r") as f:
            memory.chat_memory.messages = json.load(f)
    except (json.JSONDecodeError, FileNotFoundError):
        print("Error loading conversation history. Starting fresh.")

print("ConversationBufferWindowMemory Experiment. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Save the conversation history to the file
try:
    with open(history_file, "w") as f:
        json.dump(memory.chat_memory.messages, f)  # Save chat_memory.messages
    print(f"\nConversation history saved to {history_file}")
except Exception as e:
    print(f"Error saving conversation history: {e}")

# Print the saved memory
saved_memory = conversation.memory.load_memory_variables({})
print("\n=== Saved Memory (Last 2 Exchanges) ===")
print(saved_memory.get("history", "No memory found."))


Error loading conversation history. Starting fresh.
ConversationBufferWindowMemory Experiment. Type 'exit' to quit.
You: hello
Bot:  Hello there! It's great to chat with you. How can I assist you today?
You: my name is juan
Bot:  Hi Juan, it's nice to meet you. Is there something specific you would like to talk about?
You: exit
Error saving conversation history: Object of type HumanMessage is not JSON serializable

=== Saved Memory (Last 2 Exchanges) ===
Human: hello
AI:  Hello there! It's great to chat with you. How can I assist you today?
Human: my name is juan
AI:  Hi Juan, it's nice to meet you. Is there something specific you would like to talk about?


In [47]:
import os
import json
import atexit
from langchain.memory import ConversationBufferMemory  # or ConversationBufferWindowMemory, but note the sliding window
from langchain.chains import ConversationChain
from langchain.schema import HumanMessage, AIMessage

# Helper functions to convert message objects to/from dicts
def serialize_message(message):
    if isinstance(message, HumanMessage):
        return {"role": "human", "content": message.content}
    elif isinstance(message, AIMessage):
        return {"role": "ai", "content": message.content}
    else:
        return {"role": "unknown", "content": str(message)}

def deserialize_message(message_dict):
    role = message_dict.get("role")
    content = message_dict.get("content")
    if role == "human":
        return HumanMessage(content=content)
    elif role == "ai":
        return AIMessage(content=content)
    else:
        return HumanMessage(content=content)

# File to store conversation history
history_file = "conversation_history.json"

# Initialize memory – using full ConversationBufferMemory to keep all messages
memory = ConversationBufferMemory(return_messages=True)
# If you want a sliding window, use ConversationBufferWindowMemory(k=10, return_messages=True)
# Note: a sliding window only keeps the last 10 messages
# memory = ConversationBufferWindowMemory(k=10, return_messages=True)

# Load history if the file exists
if os.path.exists(history_file):
    try:
        with open(history_file, "r") as f:
            loaded_data = json.load(f)
        if "history" in loaded_data:
            # IMPORTANT: assign loaded messages to the chat_memory.messages attribute
            memory.chat_memory.messages = [deserialize_message(m) for m in loaded_data["history"]]
        else:
            memory.chat_memory.messages = [deserialize_message(m) for m in loaded_data]
        print("Loaded conversation history successfully.")
    except Exception as e:
        print("Error loading conversation history. Starting fresh.", e)
else:
    print("No existing history found. Starting fresh.")

# For debugging: print current loaded messages
print("Current conversation history:")
for msg in memory.chat_memory.messages:
    print(serialize_message(msg))

# Ensure you have defined your LLM (e.g., using OpenAI)
# from langchain.llms import OpenAI
# llm = OpenAI(temperature=0)
# Create your conversation chain
conversation = ConversationChain(llm=llm, memory=memory)

# Function to save history on exit
def save_history():
    try:
        with open(history_file, "w") as f:
            json.dump({"history": [serialize_message(m) for m in memory.chat_memory.messages]}, f, indent=2)
        print(f"\nConversation history saved to {history_file}")
    except Exception as e:
        print(f"Error saving conversation history: {e}")

atexit.register(save_history)

print("Chat session started. Type 'exit' to quit.")

while True:
    user_input = input("You: ")
    if user_input.strip().lower() == "exit":
        break
    response = conversation.predict(input=user_input)
    print("Bot:", response)

# Optionally call save_history() explicitly if not relying on atexit
save_history()

# Debug: print the entire memory (or last few messages)
print("\n=== Conversation Memory ===")
for msg in memory.chat_memory.messages:
    print(serialize_message(msg))


Loaded conversation history successfully.
Current conversation history:
{'role': 'human', 'content': 'my name is?'}
{'role': 'ai', 'content': ' I do not have access to your personal information, so I do not know your name. Is there something else you would like to talk about?'}
{'role': 'human', 'content': 'i am juna'}
{'role': 'ai', 'content': ' Nice to meet you, Juna! My name is AI and I am here to chat with you. Is there something specific you would like to talk about?'}
{'role': 'human', 'content': 'mi dog is carl'}
{'role': 'ai', 'content': ' I am not sure what you are trying to say. Are you asking if I know someone named Carl who has a dog named Mi?'}
{'role': 'human', 'content': 'i have a dog called carl'}
{'role': 'ai', 'content': " Oh, that's interesting! What breed is Carl? And how long have you had him?"}
{'role': 'human', 'content': 'do you remember my dog?'}
{'role': 'ai', 'content': ' I do not know your dog specifically, but I remember that you mentioned having a dog name

KeyboardInterrupt: Interrupted by user